In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib inline
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from collections import Counter
import itertools
import json
import sys
# sys.path.append("..")
from utils import data_proc_tools as dpt
from utils import plot_tools as pt
from utils.custom_metrics import recall, precision, binary_accuracy
from utils.custom_metrics import recall_np, precision_np, binary_accuracy_np, multilabel_confusion_matrix
from utils.rnn_textsum_models import Seq2SeqAtt
import random
random.seed(42)
random_state=1000
pd.set_option('display.max_colwidth', -1)
import pylab

pylab.rcParams['figure.figsize'] = (8.0, 10.0)

Using TensorFlow backend.
W1210 10:34:36.688496 139976365537024 __init__.py:321] Limited tf.compat.v2.summary API due to missing TensorBoard installation.
W1210 10:34:36.916290 139976365537024 __init__.py:321] Limited tf.compat.v2.summary API due to missing TensorBoard installation.
W1210 10:34:37.087058 139976365537024 __init__.py:352] Limited tf.summary API due to missing TensorBoard installation.
W1210 10:34:37.731662 139976365537024 __init__.py:321] Limited tf.compat.v2.summary API due to missing TensorBoard installation.


In [3]:
dir = '/vol/medic02/users/ag6516/radiology_report_summarisation/'
data_dir = dir + 'data/'

aug = 'aug'

model_output_dir = dir + 'trained_models/att_seq2seq/'

train_df = pd.read_pickle(data_dir + 'train/{}_train.pkl'.format(aug))
val_df = pd.read_pickle(data_dir + 'val/val.pkl')

## Load and prepare sequence data for training

In [4]:
train_df.head()

,examid,report,all_mesh,single_mesh
0,CXR1000_IM-0003,"[increased, opacity, within, right, upper, lobe, possible, mass, associated, area, atelectasis, focal, consolidation, ., cardiac, silhouette, within, normal, limits, ., opacity, left, midlung, overlying, posterior, left, 5th, rib, may, represent, focal, airspace, disease, ., increased, opacity, right, upper, lobe, associated, atelectasis, may, represent, focal, consolidation, mass, lesion, atelectasis, ., recommend, chest, ct, evaluation, ., opacity, overlying, left, 5th, rib, may, represent, focal, airspace, disease]","[opacity, lung, lingula, opacity, lung, upper_lobe, right, pulmonary_atelectasis, upper_lobe, right]","[opacity, lung, upper_lobe, right]"
1,CXR1001_IM-0004,"[interstitial, markings, diffusely, prominent, throughout, lungs, ., heart, size, normal, ., pulmonary, normal, ., diffuse, fibrosis]","[diffuse, markings, lung, bilateral, interstitial, diffuse, prominent]","[markings, lung, bilateral, interstitial, diffuse, prominent]"
2,CXR1002_IM-0004,"[status, post, left, mastectomy, ., heart, size, normal, ., lungs, clear]",[left],[left]
3,CXR1003_IM-0005,"[heart, size, pulmonary, vascularity, appear, within, normal, limits, ., retrocardiac, soft, tissue, density, present, ., appears, air, within, suggest, represents, hiatal, hernia, ., vascular, calcification, noted, ., calcified, granuloma, seen, ., interval, development, bandlike, opacity, left, lung, base, ., may, represent, atelectasis, ., osteopenia, present, spine, ., retrocardiac, soft, tissue, density, ., appearance, suggests, hiatal, hernia, ., left, base, bandlike, opacity, ., appearance, suggests, atelectasis]","[bone_diseases_metabolic, spine, calcified_granuloma, calcinosis, blood_vessels, density, retrocardiac, opacity, lung, base, left]","[opacity, lung, base, left]"
4,CXR1004_IM-0005,"[heart, ,, pulmonary, mediastinum, within, normal, limits, ., aorta, tortuous, ectatic, ., degenerative, changes, acromioclavicular, joints, ., degenerative, changes, spine, ., ivc, identified]","[aorta, tortuous, catheters_indwelling, shoulder, bilateral, degenerative, spine, degenerative]","[shoulder, bilateral, degenerative]"


In [5]:
# prepend and append start and end tokens to mesh captions and text reports
start_token = 'start'
end_token = 'end'
unknown_token = '**unknown**'
max_mesh_length = 13 # avg. + 1std. + start + end
max_report_length = 37 # avg. + 1std. + start + end

train_df['pad_mesh_caption'] = train_df.all_mesh.apply(lambda x: dpt.pad_sequence(x, max_mesh_length, start_token, end_token))
train_df['pad_text_report'] = train_df.report.apply(lambda x: dpt.pad_sequence(x, max_report_length, start_token, end_token))

val_df['pad_mesh_caption'] = val_df.all_mesh.apply(lambda x: dpt.pad_sequence(x, max_mesh_length, start_token, end_token))
val_df['pad_text_report'] = val_df.report.apply(lambda x: dpt.pad_sequence(x, max_report_length, start_token, end_token))

## Vectorise text reports and mesh captions

In [6]:
train_mesh = list(train_df.pad_mesh_caption)
train_reports = list(train_df.pad_text_report)

# vectorize mesh captions
dpt.mesh_to_vectors(train_mesh, dicts_dir=data_dir+'dicts/', load_dicts=True, save=True, output_dir=data_dir+'train/')

# vectorise reports
dpt.reports_to_vectors(train_reports, dicts_dir=data_dir+'dicts/', load_dicts=True, save=True, output_dir=data_dir+'train/')

In [7]:
val_reports = list(val_df.pad_text_report)
val_mesh = list(val_df.pad_mesh_caption)

# vectorise val reports + mesh using the same dict as created for train
dpt.mesh_to_vectors(val_mesh, dicts_dir=data_dir+'dicts/', load_dicts=True, save=True, output_dir=data_dir+'val/')
dpt.reports_to_vectors(val_reports, dicts_dir=data_dir+'dicts/', load_dicts=True, save=True, output_dir=data_dir+'val/')

In [8]:
word_to_id, id_to_word = dpt.load_report_dicts(data_dir+'dicts/')
mesh_to_id, id_to_mesh = dpt.load_mesh_dicts(data_dir+'dicts/')

report_vocab_length = len(word_to_id)
mesh_vocab_length = len(mesh_to_id)

In [9]:
report_vocab_length, mesh_vocab_length

(1475, 128)

In [10]:
# Create arrays of indixes for input sentences, output entities and shifted output entities (t-1)
train_token_ids_array = np.load(data_dir + 'train/token_ids_array.npy')
train_mesh_ids_array = np.load(data_dir + 'train/mesh_ids_array.npy')
train_mesh_ids_array_shifted =[np.concatenate((mesh_to_id[start_token], t[:-1]), axis=None) for t in train_mesh_ids_array]
train_mesh_ids_array_shifted = np.asarray(train_mesh_ids_array_shifted)

val_token_ids_array = np.load(data_dir + 'val/token_ids_array.npy')
val_mesh_ids_array = np.load(data_dir + 'val/mesh_ids_array.npy')
val_mesh_ids_array_shifted = [np.concatenate((mesh_to_id[start_token], t[:-1]), axis=None) for t in val_mesh_ids_array]
val_mesh_ids_array_shifted = np.asarray(val_mesh_ids_array_shifted)

In [11]:
# one-hot-encode
one_hot_reports_train = dpt.one_hot_sequence(train_token_ids_array, report_vocab_length)
one_hot_mesh_train = dpt.one_hot_sequence(train_mesh_ids_array, mesh_vocab_length)
one_hot_mesh_shifted_train = dpt.one_hot_sequence(train_mesh_ids_array_shifted, mesh_vocab_length)

one_hot_reports_val = dpt.one_hot_sequence(val_token_ids_array, report_vocab_length)
one_hot_mesh_val = dpt.one_hot_sequence(val_mesh_ids_array, mesh_vocab_length)
one_hot_mesh_shifted_val = dpt.one_hot_sequence(val_mesh_ids_array_shifted, mesh_vocab_length)

In [12]:
one_hot_reports_train.shape, one_hot_mesh_train.shape, one_hot_mesh_shifted_train.shape

((5148, 37, 1475), (5148, 13, 128), (5148, 13, 128))

## Train Seq-to-Seq Model

In [13]:
input_dim = len(word_to_id)
output_dim = len(mesh_to_id)
hidden_dim = 512
encoder_emb_dim = 512
input_seq_length = max_report_length
output_seq_length = max_mesh_length
epochs = 100
optimizer = 'adam'
loss='categorical_crossentropy'
batch_size = 128

new_experiment = Seq2SeqAtt(epochs=epochs,
                               metrics=['accuracy', binary_accuracy,recall,precision],
                               optimizer=optimizer,
                               loss=loss,
                               batch_size=batch_size, 
                               input_dim=input_dim,
                               output_dim=output_dim,
                               hidden_dim=hidden_dim,
                               input_seq_length=input_seq_length,
                               output_seq_length=output_seq_length,
                               verbose=True)
new_experiment.build_model()
new_experiment.model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(128, 37, 1475)]    0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(128, 13, 128)]     0                                            
__________________________________________________________________________________________________
lstm (LSTM)                     multiple             4071424     input_1[0][0]                    
__________________________________________________________________________________________________
lstm_1 (LSTM)                   multiple             1312768     input_2[0][0]                    
                                                                 lstm[0][1]                   

In [ ]:
# create batch generators
# train_batch_generator = dpt.batch_generator_seq2seq(train_token_ids_array, report_vocab_length, train_mesh_ids_array, 
#                                                    train_mesh_ids_array_shifted, mesh_vocab_length, batch_size)

# val_batch_generator = dpt.batch_generator_seq2seq(val_token_ids_array, report_vocab_length, val_mesh_ids_array, 
#                                                    val_mesh_ids_array_shifted, mesh_vocab_length, batch_size)

In [24]:
new_experiment.run_experiment(one_hot_reports_train, one_hot_mesh_shifted_train, one_hot_mesh_train, 
                              one_hot_reports_val, one_hot_mesh_shifted_val, one_hot_mesh_val)

Train on 5148 samples, validate on 300 samples
Epoch 1/100
5148/5148 [==============================] - 57s 11ms/sample - loss: 2.5337 - accuracy: 0.5581 - binary_accuracy: 0.9944 - recall: 0.4292 - precision: 0.6965 - val_loss: 1.6994 - val_accuracy: 0.6867 - val_binary_accuracy: 0.9961 - val_recall: 0.5672 - val_precision: 0.9137
Epoch 2/100
5148/5148 [==============================] - 52s 10ms/sample - loss: 1.8231 - accuracy: 0.6285 - binary_accuracy: 0.9959 - recall: 0.5382 - precision: 0.8970 - val_loss: 1.4274 - val_accuracy: 0.6972 - val_binary_accuracy: 0.9968 - val_recall: 0.6443 - val_precision: 0.9302
Epoch 3/100
5148/5148 [==============================] - 53s 10ms/sample - loss: 1.6517 - accuracy: 0.6458 - binary_accuracy: 0.9962 - recall: 0.5586 - precision: 0.9304 - val_loss: 1.3168 - val_accuracy: 0.7141 - val_binary_accuracy: 0.9970 - val_recall: 0.6441 - val_precision: 0.9518
Epoch 4/100
5148/5148 [==============================] - 53s 10ms/sample - loss: 1.4896 - ac

5148/5148 [==============================] - 53s 10ms/sample - loss: 0.0031 - accuracy: 0.9995 - binary_accuracy: 1.0000 - recall: 0.9995 - precision: 0.9995 - val_loss: 0.9760 - val_accuracy: 0.8479 - val_binary_accuracy: 0.9978 - val_recall: 0.8399 - val_precision: 0.8661
Epoch 58/100
5148/5148 [==============================] - 53s 10ms/sample - loss: 0.0032 - accuracy: 0.9995 - binary_accuracy: 1.0000 - recall: 0.9995 - precision: 0.9995 - val_loss: 0.9778 - val_accuracy: 0.8464 - val_binary_accuracy: 0.9978 - val_recall: 0.8379 - val_precision: 0.8671
Epoch 59/100
5148/5148 [==============================] - 53s 10ms/sample - loss: 0.0038 - accuracy: 0.9994 - binary_accuracy: 1.0000 - recall: 0.9994 - precision: 0.9995 - val_loss: 0.9732 - val_accuracy: 0.8472 - val_binary_accuracy: 0.9978 - val_recall: 0.8395 - val_precision: 0.8696
Epoch 60/100
5148/5148 [==============================] - 53s 10ms/sample - loss: 0.0044 - accuracy: 0.9993 - binary_accuracy: 1.0000 - recall: 0.999

In [27]:
new_experiment.save_weights_history(model_output_dir)

## Load results of specific experiment

In [13]:
model_name = 'seq2seq_att'
model_output_dir = dir + 'trained_models/{}/'.format(model_name)

In [14]:
import pickle

emb = False
epochs = 100
hidden_dim = 512

param_fn = 'param_seq2seqatt_epochs_{}_hiddendim_{}.pkl'\
.format(epochs, hidden_dim)
params = pickle.load(open(model_output_dir + param_fn, 'rb'))

old_experiment = Seq2SeqAtt(**params)
old_experiment.build_model()
old_experiment.load_weights_history(model_output_dir)

In [15]:
# decode a one hot encoded string
def one_hot_decode(encoded_seq):
    return [np.argmax(vector) for vector in encoded_seq]

In [16]:
def strip_start_end(seq, start_token='start', end_token='end'):
    stripped_seq = []
    for s in seq:
        if s not in [start_token, end_token]:
            stripped_seq.append(s)
    return stripped_seq

In [17]:
for _ in range(10):
    sample = train_df.sample(1)
    true_mesh_caption = list(sample.all_mesh)[0]
    sample_report = list(sample.pad_text_report)[0]
    
    sample_report_ids = []
    for token in sample_report:
        if token in word_to_id.keys():
            sample_report_ids.append(word_to_id[token])
        else:
            sample_report_ids.append(word_to_id[unknown_token])
    
    sample_report_ids = np.array(sample_report_ids).reshape(1, len(sample_report_ids))
    one_hot_sample_report = dpt.one_hot_sequence(sample_report_ids, report_vocab_length)
    
    #target = predict_sequence(infenc, infdec, one_hot_sample_report, n_steps_out, n_features_out)
    target = old_experiment.predict_sequence(one_hot_sample_report)
    predicted_mesh_ids = one_hot_decode(target)
    predicted_mesh = [id_to_mesh[idx] for idx in predicted_mesh_ids]
    
    sample_report = strip_start_end(sample_report)
    predicted_mesh = strip_start_end(predicted_mesh)
    
    print('')
    print('Original report: ', sample_report)
    print('True mesh caption: ', true_mesh_caption)
    print('Predicted mesh caption: ', predicted_mesh)


Original report:  ['normal', 'heart', 'size', '.', 'anterior', 'osteophytes', 'thoracic', 'spine']
True mesh caption:  ['osteophyte', 'thoracic_vertebrae', 'anterior', 'multiple']
Predicted mesh caption:  ['osteophyte', 'thoracic_vertebrae', 'anterior', 'multiple']

Original report:  ['lungs', 'mildly', 'hyperinflated', 'flattened', 'posterior', 'diaphragm', 'increased', 'retrosternal', 'airspace', '.', 'left', 'hilar', 'calcifications', 'dense', 'left', 'lower', 'lobe', 'nodules', 'suggest', 'previous', 'granulomatous', 'process', '.', 'hyperinflated', 'lungs', ',', 'air', 'trapping', 'versus', 'inspiratory']
True mesh caption:  ['calcinosis', 'lung', 'hilum', 'left', 'density', 'lung', 'lower_lobe', 'left', 'round', 'multiple', 'diaphragm', 'posterior', 'flattened', 'right', 'prominent', 'lung', 'hyperdistention', 'mild', 'nodule', 'lung', 'lower_lobe', 'left', 'multiple']
Predicted mesh caption:  ['calcinosis', 'lung', 'hilum', 'left', 'density', 'lung', 'lower_lobe', 'left', 'roun

## Evaluate BLEU scores on all trian/val/test data

In [18]:
import nltk
from nltk.translate.bleu_score import sentence_bleu

def evaluate_model(model, df, report_vocab_length):
    actual, predicted = list(), list()
    bleu1, bleu2, bleu3, bleu4 = list(), list(), list(), list()

    for _, sample in df.iterrows():
        true_mesh_caption = sample.all_mesh
        sample_report = sample.pad_text_report

        sample_report_ids = []
        for token in sample_report:
            if token in word_to_id.keys():
                sample_report_ids.append(word_to_id[token])
            else:
                sample_report_ids.append(word_to_id[unknown_token])

        sample_report_ids = np.array(sample_report_ids).reshape(1, len(sample_report_ids))
        one_hot_sample_report = dpt.one_hot_sequence(sample_report_ids, report_vocab_length)

        #target = predict_sequence(infenc, infdec, one_hot_sample_report, n_steps_out, n_features_out)
        target = model.predict_sequence(one_hot_sample_report)
        predicted_mesh_ids = one_hot_decode(target)
        predicted_mesh = [id_to_mesh[idx] for idx in predicted_mesh_ids]

        # sample_report = strip_start_end(sample_report)
        yhat = strip_start_end(predicted_mesh)
        reference = true_mesh_caption
        
        # calculate BLEU score
        bleu1.append(sentence_bleu([reference], yhat, weights=(1.0, 0, 0, 0)))
        bleu2.append(sentence_bleu([reference], yhat, weights=(0.5, 0.5, 0, 0)))
        bleu3.append(sentence_bleu([reference], yhat, weights=(0.3, 0.3, 0.3, 0)))
        bleu4.append(sentence_bleu([reference], yhat, weights=(0.25, 0.25, 0.25, 0.25)))
    
        # store actual and predicted
        actual.append(reference)
        predicted.append(yhat)
        
    print('BLEU1: ', np.mean(bleu1)*100)
    print('BLEU2: ', np.mean(bleu2)*100)
    print('BLEU3: ', np.mean(bleu3)*100)
    print('BLEU4: ', np.mean(bleu4)*100)
    
    return actual, predicted

In [45]:
train_actual, train_predicted = evaluate_model(old_experiment, train_df, report_vocab_length)

/vol/medic02/users/ag6516/python3env/lib/python3.6/site-packages/nltk/translate/bleu_score.py:523: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/vol/medic02/users/ag6516/python3env/lib/python3.6/site-packages/nltk/translate/bleu_score.py:523: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/vol/medic02/users/ag6516/python3env/lib/python3.6/site-packages/nltk/translate/bleu_score.py:523: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Con

BLEU1:  93.91184482714614
BLEU2:  69.49247862087088
BLEU3:  60.51783875821691
BLEU4:  51.45224917943823


In [46]:
val_actual, val_predicted = evaluate_model(old_experiment, val_df, report_vocab_length)

BLEU1:  68.21554414383871
BLEU2:  26.26280134482814
BLEU3:  15.299111087191413
BLEU4:  6.9175551610530075


## Evaluate ROUGE scores on all train/val/test data

In [49]:
import rouge

evaluator = rouge.Rouge(metrics=['rouge-n', 'rouge-l', 'rouge-w'],
                       max_n=4,
                       limit_length=True,
                       length_limit=100,
                       length_limit_type='words',
                       apply_avg='Avg',
                       apply_best='Best',
                       alpha=0.5, # Default F1_score
                       weight_factor=1.2,
                       stemming=True)

In [50]:
def prepare_results(p, r, f):
    return '\t{}:\t{}: {:5.2f}\t{}: {:5.2f}\t{}: {:5.2f}'.format(metric, 'P', 100.0 * p, 'R', 100.0 * r, 'F1', 100.0 * f)

In [51]:
train_hypotheses = [' '.join(p) for p in train_predicted]
train_references = [' '.join(a) for a in train_actual]

scores = evaluator.get_scores(train_hypotheses, train_references)

for metric, results in sorted(scores.items(), key=lambda x: x[0]):
    print(prepare_results(results['p'], results['r'], results['f']))

	rouge-1:	P: 99.56	R: 94.74	F1: 96.54
	rouge-2:	P: 76.61	R: 71.54	F1: 73.40
	rouge-3:	P: 70.41	R: 65.06	F1: 66.98
	rouge-4:	P: 62.87	R: 57.22	F1: 59.21
	rouge-l:	P: 99.57	R: 95.39	F1: 97.03
	rouge-w:	P: 99.38	R: 72.31	F1: 81.98


In [52]:
val_hypotheses = [' '.join(p) for p in val_predicted]
val_references = [' '.join(a) for a in val_actual]

scores = evaluator.get_scores(val_hypotheses, val_references)

for metric, results in sorted(scores.items(), key=lambda x: x[0]):
    print(prepare_results(results['p'], results['r'], results['f']))

	rouge-1:	P: 77.44	R: 73.07	F1: 73.87
	rouge-2:	P: 33.00	R: 29.79	F1: 30.16
	rouge-3:	P: 21.69	R: 19.61	F1: 19.67
	rouge-4:	P: 13.23	R: 11.88	F1: 11.96
	rouge-l:	P: 77.97	R: 74.26	F1: 75.09
	rouge-w:	P: 73.58	R: 59.60	F1: 63.71
